In [28]:
from BS_pricer import BS_pricer
from Parameters import Option_param
from Processes import Diffusion_process
from Merton_pricer import Merton_pricer
from Processes import Diffusion_process, Merton_process, VG_process, NIG_process
from VG_pricer import VG_pricer
from NIG_pricer import NIG_pricer

import numpy as np
import scipy.stats as ss
from scipy.integrate import quad
from functools import partial

import matplotlib.pyplot as plt


In [18]:
S0 = 100.0  # spot stock price
K = 100.0  # strike
T = 1.0  # maturity
r = 0.1  # risk free rate
sig = 0.2  # diffusion coefficient or volatility
N = 10000000  # number of generated random variables
k = np.log(K / S0)  # log moneyness
theta = -0.11  # drift of the Brownian motion
sigma = 0.2  # volatility of the Brownian motion
kappa = 0.3  # variance of the IG process
lam = T**2 / kappa  # scale
mu_s = T / lam  # scaled mu

In [19]:
call = BS_pricer.BlackScholes("call", S0, K, T, r, sig)
put = BS_pricer.BlackScholes("put", S0, K, T, r, sig)
print("Call price: ", call)
print("Put price: ", put)

Call price:  13.269676584660893
Put price:  3.753418388256833


In [20]:
print(call)
print(put + S0 - K * np.exp(-r * T))

13.269676584660893
13.269676584660886


<a id='sec3'></a>
## Monte Carlo method

I'm going to simulate the random variables: 

$$ S_T^i = S_0 e^{(r -\frac{1}{2}\sigma^2)T + \sigma W_{T}^i} $$

for $1 \leq i \leq N$.    
Then use the approximation for a call option:

$$ \mathbb{E}^{\mathbb{Q}}\biggl[ (S_T - K)^+ \bigg| S_0 \biggr] \; 
\approx \; \frac{1}{N} \sum_{i=1}^N (S_T^i - K)^+
$$

For a put option I use this payoff $(K - S_T )^+$ inside the expectation.

In [21]:
np.random.seed(seed=44)  # seed for random number generation
N = 10000000  # Number of random variables

W = ss.norm.rvs((r - 0.5 * sig**2) * T, np.sqrt(T) * sig, N)
S_T = S0 * np.exp(W)

call = np.mean(np.exp(-r * T) * np.maximum(S_T - K, 0))
put = np.mean(np.exp(-r * T) * np.maximum(K - S_T, 0))
call_err = ss.sem(np.exp(-r * T) * np.maximum(S_T - K, 0))  # standard error
put_err = ss.sem(np.exp(-r * T) * np.maximum(K - S_T, 0))  # standard error

In [22]:
print("Call price: {}, with error: {}".format(call, call_err))
print("Put price: {}, with error: {}".format(put, put_err))

Call price: 13.26333800663666, with error: 0.005093638687208466
Put price: 3.7553894006350084, with error: 0.002214066662789331


\begin{equation}
X_t = \mu t + \sigma W_t + \sum_{i=1}^{N_t} Y_i, 
\end{equation}

where $N_t$ is a Poisson random variable representing the number of jumps of $X_t$ up to time $t$, and $Y_i \sim \mathcal{N}(\alpha, \xi^2)$ is the size of each jump.

In [23]:
opt_param = Option_param(S0=100, K=100, T=1, exercise="European", payoff="call")
opt_param_p = Option_param(S0=100, K=100, T=1, exercise="European", payoff="put")

Merton_param = Merton_process(r=0.1, sig=0.2, lam=0.8, muJ=0, sigJ=0.5)
Merton = Merton_pricer(opt_param, Merton_param)

In [24]:
Merton.MC(1000000, Err=True, Time=True)

(array([22.05548142]), array([0.05731805]), 26.648557662963867)

In [25]:
# Creates the object with the parameters of the option
opt_param = Option_param(S0=100, K=100, T=1, exercise="European", payoff="call")
# Creates the object with the parameters of the process
VG_param = VG_process(r=0.1, sigma=0.2, theta=-0.1, kappa=0.3)
# Creates the VG pricer
VG = VG_pricer(opt_param, VG_param)

In [26]:
VG.MC(20000000, Err=True, Time=True)

(array([13.40104561]), array([0.00348222]), 2.314268112182617)

In [ ]:
w = (1 - np.sqrt(1 - 2 * theta * kappa - kappa * sigma**2)) / kappa  # Martingale correction
dev_X = np.sqrt(sigma**2 + theta**2 * kappa)  # std dev NIG process

In [32]:
opt_param = Option_param(exercise ="European", payoff="call")
NIG_param = NIG_process()
NIG = NIG_pricer(opt_param, NIG_param)

In [33]:
NIG.MC(20000000, Err=True, Time=True)

(1.9964473756889853, array([0.00053435]), 2.562917947769165)